# Second Layer: Genesis of Non-Gaussianity

In [1]:
import sympy as sp

from symdl.utils import pull_sums_out_front, sum_kronecker_contract, wilds, wild_subs
from symdl.random import ExpVal, RandomIndexedBase
from symdl.gaussian import GaussianIndexedBase, wick_contraction
from symdl.nn import ZLayer

In [2]:
i = sp.symbols('i1:5', integer=True) # neouron index
alpha = sp.symbols('alpha1:5', integer=True) # sample index

# Input
sigma1 = RandomIndexedBase('sigma^(1)')
n1 = sp.symbols('n1', integer=True)

# Layer param
b = GaussianIndexedBase('b^(2)')
W = GaussianIndexedBase('W^(2)')
C_b, C_w = sp.symbols('C_b^(2) C_W^(2)', positive=True)

In [3]:
z2 = ZLayer(W, b, sigma1, n1)
z2[i[0], alpha[0]]

Sum(W^(2)[i1, k_1]*sigma^(1)[k_1, alpha1], (k_1, 1, n1)) + b^(2)[i1]

The joint distribution of preactivations in the ﬁrst and second layers is:

$$p\left(z^{(2)}, z^{(1)} \mid \mathcal{D} \right) = p\left(z^{(2)} \mid z^{(1)} \right) p\left(z^{(1)} \mid \mathcal{D} \right)
$$

The marginal distribution of the second-layer preactivations can then be obtained by **marginalizing over** or **integrating out** the ﬁrst-layer
preactivations:

$$p\left(z^{(2)} \mid \mathcal{D} \right) = \int \left[ \prod_{i, \alpha} dz^{(1)}_{i;\alpha} \right] p\left(z^{(2)} \mid z^{(1)} \right) p\left(z^{(1)} \mid \mathcal{D} \right)
$$

- The ﬁrst-layer marginal distribution is:

$$p\left(z^{(1)} \mid \mathcal{D} \right) = \frac{1}{\left| 2\pi G^{(1)} \right|^{n_1 / 2}} \exp\left( -\frac{1}{2} \sum_{i=1}^{n_1} \sum_{\alpha_1, \alpha_2 \in \mathcal{D}} G^{\alpha_1 \alpha_2}_{(1)} z^{(1)}_{i; \alpha_1} z^{(1)}_{i; \alpha_2} \right)
$$

- The conditional distribution is:

$$p\left(z^{(2)} \mid z^{(1)} \right) = \frac{1}{\left| 2\pi \widehat{G}^{(2)} \right|^{n_2/2}} \exp\left( -\frac{1}{2} \sum_{i=1}^{n_2} \sum_{\alpha_1, \alpha_2 \in \mathcal{D}} \widehat{G}^{\alpha_1 \alpha_2}_{(2)} z^{(2)}_{i; \alpha_1} z^{(2)}_{i; \alpha_2} \right)$$

where $\widehat{G}^{(2)}_{\alpha_1 \alpha_2} \equiv C^{(2)}_b + C^{(2)}_W \frac{1}{n_1} \sum_{j=1}^{n_1} \sigma^{(1)}_{j; \alpha_1} \sigma^{(1)}_{j; \alpha_2}$ is *stochastic* second-layer metric



In [4]:
zz: sp.Expr = sp.expand(z2[i[0], alpha[0]] * z2[i[1], alpha[1]])
zz = pull_sums_out_front(zz)
Ezz: sp.Expr = ExpVal(zz)
Ezz

𝔼[b^(2)[i1]*b^(2)[i2]] + Sum(𝔼[W^(2)[i1, k_2]*b^(2)[i2]*sigma^(1)[k_2, alpha1]], (k_2, 1, n1)) + Sum(𝔼[W^(2)[i2, k_3]*b^(2)[i1]*sigma^(1)[k_3, alpha2]], (k_3, 1, n1)) + Sum(𝔼[W^(2)[i1, k_2]*W^(2)[i2, k_3]*sigma^(1)[k_2, alpha1]*sigma^(1)[k_3, alpha2]], (k_2, 1, n1), (k_3, 1, n1))